In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Pernambuco - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego,Pernambuco - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.297418,34.266080,8.237381e+07,5.574893e+06,8.646979,7.571061e+07,0.688685,8.294029,99.743
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.298912,34.126272,8.248102e+07,5.580364e+06,8.650960,7.575437e+07,0.689009,8.288006,86.997
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.300346,34.414272,8.258823e+07,5.585835e+06,8.654941,7.579812e+07,0.689334,8.281983,83.033
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.301729,34.576094,8.269544e+07,5.591305e+06,8.658922,7.584188e+07,0.689658,8.275960,82.302
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.302368,34.890197,8.280265e+07,5.596776e+06,8.662903,7.588563e+07,0.689983,8.269937,80.217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478890,26.779678,NaN,NaN,NaN,NaN,NaN,NaN,164.137
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.478683,26.996501,NaN,NaN,NaN,NaN,NaN,NaN,168.356
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.476012,26.919241,NaN,NaN,NaN,NaN,NaN,NaN,172.421
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.473486,27.026250,NaN,NaN,NaN,NaN,NaN,NaN,165.411


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.011454,-1.188910,-1.754154,-0.181030,-2.511326,-2.321118,-2.100972,-0.864836
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.964614,-1.195377,-1.734705,-0.148206,-2.462275,-2.280399,-2.067858,-0.866889
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.919667,-1.182056,-1.715257,-0.115382,-2.413223,-2.239680,-2.034743,-0.868942
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.876288,-1.174571,-1.695809,-0.082559,-2.364172,-2.198961,-2.001628,-0.870995
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.856283,-1.160042,-1.676361,-0.049735,-2.315120,-2.158242,-1.968514,-0.873048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.720800,-1.281757,1.016740,-1.416488,-0.070259,0.355961,1.324101,1.183158
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.802101,-1.271596,1.004554,-1.402870,-0.093306,0.336941,1.314733,1.181904
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.813469,-1.268136,0.992368,-1.389252,-0.116354,0.317922,1.305366,1.180650
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.843043,-1.270527,0.980182,-1.375634,-0.139402,0.298902,1.295999,1.179397


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      76.341
1      65.040
2      88.306
3      74.323
4      78.114
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Pernambuco - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.011454,-1.188910,-1.754154,-0.181030,-2.511326,-2.321118,-2.100972,-0.864836
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.964614,-1.195377,-1.734705,-0.148206,-2.462275,-2.280399,-2.067858,-0.866889
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.919667,-1.182056,-1.715257,-0.115382,-2.413223,-2.239680,-2.034743,-0.868942
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.876288,-1.174571,-1.695809,-0.082559,-2.364172,-2.198961,-2.001628,-0.870995
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.856283,-1.160042,-1.676361,-0.049735,-2.315120,-2.158242,-1.968514,-0.873048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.354413,-0.165538,1.170965,-1.488331,0.445555,0.786479,1.391411,1.357017
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.374652,-0.236996,1.172520,-1.494202,0.438851,0.781348,1.390568,1.349817
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.400582,-0.309267,1.174075,-1.500073,0.432147,0.776217,1.389726,1.342617
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.422300,-0.382346,1.175630,-1.505943,0.425442,0.771086,1.388883,1.335417


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       76.341
1       65.040
2       88.306
3       74.323
4       78.114
        ...   
157    143.388
158    188.227
159    143.049
160    148.068
161    133.872
Name: Pernambuco - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pernambuco - value,Pernambuco - Produção de Cimento (t),Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - Desemprego
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.276351,1.507092,0.863757,-0.284837,0.991047,1.042288,0.483520,0.905318
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.291616,1.497914,0.879821,-0.344037,0.974598,1.037752,0.491009,0.926343
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.299215,1.471401,0.895884,-0.403236,0.958148,1.033215,0.498499,0.947367
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.301690,1.429249,0.911948,-0.462436,0.941699,1.028678,0.505988,0.968392
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.304954,1.390164,0.928011,-0.521635,0.925249,1.024142,0.513478,0.989417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.720800,-1.281757,1.016740,-1.416488,-0.070259,0.355961,1.324101,1.183158
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.802101,-1.271596,1.004554,-1.402870,-0.093306,0.336941,1.314733,1.181904
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.813469,-1.268136,0.992368,-1.389252,-0.116354,0.317922,1.305366,1.180650
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.843043,-1.270527,0.980182,-1.375634,-0.139402,0.298902,1.295999,1.179397


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[749950055, 187316805, 2673190820, 3342770720, 3744122902, 1879719278, 2011505996, 4130459831, 1552415303, 2884130106, 1539503073, 231154906, 851921697, 4102064029, 2340451304, 1584474939, 2089162784, 3109171043, 2765271432, 3592140404, 3075304711, 1887268803, 1178359162, 1012406196, 413795424, 2907082380, 3388584673, 1994981069, 254157658, 1268289205, 1912688175, 1737745679, 860902414, 3761034984, 341854454, 3185945999, 2473977165, 973101683, 3866062302, 986460286, 4017358431, 1684639182, 1185050330, 397274462, 465226814, 2717854353, 2490970917, 1533410966, 3831537298, 2578247319]


Step: 0 ___________________________________________
val_loss: 141.67181396484375
winner_seed: 749950055


Step: 1 ___________________________________________
val_loss: 188.15814208984375


Step: 2 ___________________________________________
val_loss: 177.01951599121094


Step: 3 ___________________________________________
val_loss: 149.7744598388672


Step: 4 ___________________________________________
val

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 118ms/step - loss: 29533.4473 - val_loss: 2106.0745
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 2301.8162 - val_loss: 2182.2742
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 2139.2693 - val_loss: 2072.7822
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 2071.9902 - val_loss: 2790.5413
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 2569.4885 - val_loss: 1868.4049
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 1800.1300 - val_loss: 1827.5845
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 1520.1406 - val_loss: 1284.2603
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 1330.0494 - val_loss: 1258.2375
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 1657.1952 - val_loss: 2840.3423
Epoch 10/10000
4/4 [===========================

4/4 [==============================] - 0s 8ms/step - loss: 431.9189 - val_loss: 357.9008
Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 345.1409 - val_loss: 312.9467
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 352.3193 - val_loss: 707.3586
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 391.2085 - val_loss: 458.5883
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 350.5143 - val_loss: 350.4532
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 338.2975 - val_loss: 679.7341
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 426.0487 - val_loss: 306.1155
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 336.4031 - val_loss: 451.7549
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 325.8751 - val_loss: 349.3792
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 288.

4/4 [==============================] - 0s 10ms/step - loss: 213.9935 - val_loss: 242.3945
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 224.4224 - val_loss: 301.5763
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 242.5814 - val_loss: 472.2433
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 346.5037 - val_loss: 270.1456
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 241.3791 - val_loss: 330.1444
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 219.4396 - val_loss: 277.7237
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 210.1415 - val_loss: 415.4179
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 432.3972 - val_loss: 320.0628
Epoch 165/10000
4/4 [==============================] - 0s 8ms/step - loss: 233.1411 - val_loss: 294.0624
Epoch 166/10000
4/4 [==============================] - 0s 8ms/step

Epoch 235/10000
4/4 [==============================] - 0s 10ms/step - loss: 185.4104 - val_loss: 360.5544
Epoch 236/10000
4/4 [==============================] - 0s 10ms/step - loss: 241.9405 - val_loss: 473.3849
Epoch 237/10000
4/4 [==============================] - 0s 10ms/step - loss: 254.2767 - val_loss: 359.8992
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 269.8575 - val_loss: 324.8505
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 224.7387 - val_loss: 301.0472
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 185.8130 - val_loss: 315.8715
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 182.5440 - val_loss: 340.9966
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 192.5074 - val_loss: 254.6861
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 202.6740 - val_loss: 281.2248
Epoch 244/10000
4/4 [==============================]

4/4 [==============================] - 0s 9ms/step - loss: 279.4596 - val_loss: 307.7161
Epoch 313/10000
4/4 [==============================] - 0s 10ms/step - loss: 401.4492 - val_loss: 322.9892
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 341.8769 - val_loss: 471.2632
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 323.0061 - val_loss: 345.0813
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step - loss: 380.1842 - val_loss: 299.1804
Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 358.9044 - val_loss: 321.0199
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 404.0399 - val_loss: 436.7921
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 369.1837 - val_loss: 276.0877
Epoch 320/10000
4/4 [==============================] - 0s 10ms/step - loss: 292.7862 - val_loss: 344.8132
Epoch 321/10000
4/4 [==============================] - 0s 10ms/ste

...

Epoch 700/10000
4/4 [==============================] - 0s 9ms/step - loss: 211.8935 - val_loss: 286.2596
Epoch 701/10000
4/4 [==============================] - 0s 10ms/step - loss: 240.8334 - val_loss: 391.6547
Epoch 702/10000
4/4 [==============================] - 0s 10ms/step - loss: 206.1125 - val_loss: 279.6511
Epoch 703/10000
4/4 [==============================] - 0s 11ms/step - loss: 179.5779 - val_loss: 267.0880
Epoch 704/10000
4/4 [==============================] - 0s 10ms/step - loss: 196.3468 - val_loss: 225.4786
Epoch 705/10000
4/4 [==============================] - 0s 10ms/step - loss: 187.4586 - val_loss: 233.3564
Epoch 706/10000
4/4 [==============================] - 0s 10ms/step - loss: 179.7770 - val_loss: 219.4184
Epoch 707/10000
4/4 [==============================] - 0s 9ms/step - loss: 179.6875 - val_loss: 223.3352
Epoch 708/10000
4/4 [==============================] - 0s 8ms/step - loss: 161.2900 - val_loss: 279.8926
Epoch 709/10000
4/4 [============================

Epoch 778/10000
4/4 [==============================] - 0s 10ms/step - loss: 192.6266 - val_loss: 241.7059
Epoch 779/10000
4/4 [==============================] - 0s 9ms/step - loss: 160.7790 - val_loss: 227.6129
Epoch 780/10000
4/4 [==============================] - 0s 9ms/step - loss: 160.9227 - val_loss: 389.4113
Epoch 781/10000
4/4 [==============================] - 0s 9ms/step - loss: 191.8152 - val_loss: 271.8156
Epoch 782/10000
4/4 [==============================] - 0s 9ms/step - loss: 179.8186 - val_loss: 302.9658
Epoch 783/10000
4/4 [==============================] - 0s 9ms/step - loss: 188.5251 - val_loss: 350.0336
Epoch 784/10000
4/4 [==============================] - 0s 10ms/step - loss: 186.6864 - val_loss: 290.3640
Epoch 785/10000
4/4 [==============================] - 0s 10ms/step - loss: 170.5875 - val_loss: 271.8914
Epoch 786/10000
4/4 [==============================] - 0s 10ms/step - loss: 210.7749 - val_loss: 251.9835
Epoch 787/10000
4/4 [==============================

Epoch 856/10000
4/4 [==============================] - 0s 8ms/step - loss: 160.6332 - val_loss: 440.7994
Epoch 857/10000
4/4 [==============================] - 0s 8ms/step - loss: 204.3513 - val_loss: 293.0070
Epoch 858/10000
4/4 [==============================] - 0s 9ms/step - loss: 207.0149 - val_loss: 378.1850
Epoch 859/10000
4/4 [==============================] - 0s 9ms/step - loss: 177.6265 - val_loss: 264.0965
Epoch 860/10000
4/4 [==============================] - 0s 9ms/step - loss: 146.2899 - val_loss: 316.5950
Epoch 861/10000
4/4 [==============================] - 0s 8ms/step - loss: 152.8170 - val_loss: 281.9709
Epoch 862/10000
4/4 [==============================] - 0s 9ms/step - loss: 167.8767 - val_loss: 308.8547
Epoch 863/10000
4/4 [==============================] - 0s 9ms/step - loss: 158.2002 - val_loss: 291.6822
Epoch 864/10000
4/4 [==============================] - 0s 9ms/step - loss: 152.4420 - val_loss: 277.5443
Epoch 865/10000
4/4 [==============================] - 

Epoch 934/10000
4/4 [==============================] - 0s 8ms/step - loss: 157.2684 - val_loss: 179.0422
Epoch 935/10000
4/4 [==============================] - 0s 8ms/step - loss: 136.5987 - val_loss: 318.3259
Epoch 936/10000
4/4 [==============================] - 0s 8ms/step - loss: 153.2330 - val_loss: 210.1474
Epoch 937/10000
4/4 [==============================] - 0s 9ms/step - loss: 169.3434 - val_loss: 289.3421
Epoch 938/10000
4/4 [==============================] - 0s 9ms/step - loss: 163.2827 - val_loss: 170.8130
Epoch 939/10000
4/4 [==============================] - 0s 8ms/step - loss: 179.2161 - val_loss: 204.8160
Epoch 940/10000
4/4 [==============================] - 0s 9ms/step - loss: 178.1088 - val_loss: 282.9533
Epoch 941/10000
4/4 [==============================] - 0s 9ms/step - loss: 142.0016 - val_loss: 204.0957
Epoch 942/10000
4/4 [==============================] - 0s 9ms/step - loss: 146.1482 - val_loss: 352.3014
Epoch 943/10000
4/4 [==============================] - 

Epoch 1012/10000
4/4 [==============================] - 0s 10ms/step - loss: 129.3526 - val_loss: 367.0534
Epoch 1013/10000
4/4 [==============================] - 0s 9ms/step - loss: 149.2137 - val_loss: 292.6814
Epoch 1014/10000
4/4 [==============================] - 0s 9ms/step - loss: 153.5493 - val_loss: 336.2702
Epoch 1015/10000
4/4 [==============================] - 0s 9ms/step - loss: 131.2302 - val_loss: 292.5858
Epoch 1016/10000
4/4 [==============================] - 0s 8ms/step - loss: 140.6567 - val_loss: 281.7155
Epoch 1017/10000
4/4 [==============================] - 0s 8ms/step - loss: 132.8276 - val_loss: 321.9234
Epoch 1018/10000
4/4 [==============================] - 0s 9ms/step - loss: 134.0733 - val_loss: 324.2264
Epoch 1019/10000
4/4 [==============================] - 0s 8ms/step - loss: 112.7942 - val_loss: 351.5913
Epoch 1020/10000
4/4 [==============================] - 0s 8ms/step - loss: 118.1970 - val_loss: 281.7179
Epoch 1021/10000
4/4 [=======================

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,154.908585,154.861938,155.216034,155.160416,155.271866,155.599197,156.190765,156.147232,156.563675,156.279678,156.63887,157.215988,158.410568,158.837357,158.750732,159.513046,159.877319,160.339401,161.011093,160.656311,161.418716,161.958603,160.998398,161.47226,161.037552,162.873489,162.645142,165.014893,163.743729,165.401718
Target,131.464,151.37,154.276,153.331,152.6,136.567,144.855,126.352,138.148,129.978,109.083,147.44,132.5,158.533,133.813,155.827,146.197,122.278,149.983,123.483,125.104,129.633,138.233,109.657,128.809,140.408,134.634,189.04,166.406,155.456
Error,23.44458,3.491943,0.940033,1.829422,2.67186,19.032196,11.33577,29.795235,18.41568,26.301682,47.55587,9.775986,25.910568,0.304352,24.937729,3.68605,13.680313,38.061401,11.028091,37.173309,36.31472,32.325607,22.765396,51.815262,32.228546,22.465485,28.011139,24.025101,2.662277,9.945724


In [38]:
display(mae)
display(mape)

20.39771

0.15724553

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined